In [39]:
import pandas as pd

import osmosys.osmosys
import activityinfo
from activityinfo import Client
from activityinfo import FormProcessing
import importlib
import model.modelAI
from activityinfo.id import generate_id
import json
from requests.exceptions import HTTPError
import os.path

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo.FormProcessing)
importlib.reload(model.modelAI)

<module 'model.modelAI' from 'D:\\Proyectos\\OSMOSYS_2022\\IntegraciónAI\\AI_OsmosysIntegrationScripts\\model\\modelAI.py'>

In [40]:
## parameters
indicatorCodeAI = 'PRPI1_01'
month = 'JUNIO'
month_number = '2022-06'
year = 2022
test = True
print(
    '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')


--------------------------------------PRPI1_01------------------------------------------------------


In [41]:
# busco matchs
subformDf = osmosys.osmosys.getMatchSubforms(indicatorCodeAI)
## print(subformDf)
indicatorIdsOsmosys = subformDf.osmosys_indicador_id.unique()
subformSeries = subformDf.iloc[0]
formIdAI = subformSeries.form_id
parentFormIdAI = subformSeries.parent_form_id
indicatorIdAI = subformSeries.indicator_id
print("formIdAI: " + formIdAI)
print("parentFormIdAI: " + parentFormIdAI)
print("indicatorIdAI: " + indicatorIdAI)
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))

formIdAI: clsgys9l4q3ommxdv
parentFormIdAI: cr62ru9l4q3ommxdt
indicatorIdAI: cjo64mykylzqu4p4r
indicatorIdsOsmosys: [85 35]


In [42]:
## consulto los datos reportados
reportedDf = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                                month=month, year=year)
print('reported data: ' + str(reportedDf.shape[0]))


reported data: 6


In [43]:
## obtengo los parents de AI
parentFormsIds = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDf)

In [44]:
## 2 fuentes por separado indicador 35
indicatorIdsOsmosys35 = [35]
## contruyo las estructuras de datos
changesList = []
newIds = []
for index, row in parentFormsIds.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys35, cantonCode=cantonCode)

    poblacion_meta = ['Refugiados/as y migrantes', 'Comunidad de acogida']

    PRPI1_01_RM_NA = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    PRPI1_01_RM_NN = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    PRPI1_01_RM_OTR = int(df.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)
    dfDiversidad = osmosys.osmosys.getRefLgbtiDiscapacitadosValues(year=year, month=month, orgOsmosys=orgAcron,
                                                                   indicatorsIdsOmosys=indicatorIdsOsmosys35,
                                                                   cantonCode=cantonCode)
    PRPI1_01_RM_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)
    PRPI1_01_RM_DS = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'DISCAPACITADOS') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)

    dfCa = osmosys.osmosys.getCAValues(year=year, month=month, orgOsmosys=orgAcron,
                                       indicatorsIdsOmosys=indicatorIdsOsmosys35, cantonCode=cantonCode)
    PRPI1_01_CA_NA = int(dfCa.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    PRPI1_01_CA_NN = int(dfCa.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    PRPI1_01_CA_OTR = int(dfCa.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)

    PRPI1_01_CA_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'ECUADOR')].iloc[0].value_a)
    PRPI1_01_CA_DS = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'DISCAPACITADOS') & (
            dfDiversidad['country_of_origin'] == 'ECUADOR')].iloc[0].value_a)
    PRPI1_01_CUAL = commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                                               indicatorsIdsOmosys=indicatorIdsOsmosys35).iloc[
        0].value_a

    subform = model.modelAI.SubFormPRPI1_01_35(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        covid='No',
        espacion_apoyo='No',
        poblacion_meta=poblacion_meta,
        modalidad_impl='cmo12q1kykwwosjka',  ## servicio
        PRPI1_01_RM_NA=PRPI1_01_RM_NA,
        PRPI1_01_RM_NN=PRPI1_01_RM_NN,
        PRPI1_01_RM_OTR=PRPI1_01_RM_OTR,
        PRPI1_01_RM_LGBT=PRPI1_01_RM_LGBT,
        PRPI1_01_RM_DS=PRPI1_01_RM_DS,
        PRPI1_01_CA_NA=PRPI1_01_CA_NA,
        PRPI1_01_CA_NN=PRPI1_01_CA_NN,
        PRPI1_01_CA_OTR=PRPI1_01_CA_OTR,
        PRPI1_01_CA_LGBT=PRPI1_01_CA_LGBT,
        PRPI1_01_CA_DS=PRPI1_01_CA_DS,
        PRPI1_01_CUAL=PRPI1_01_CUAL
    )
    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
print('changes to charge: ' + str(len(changesList)))

changes to charge: 3


In [45]:
indicatorIdsOsmosys85 = [85]
## contruyo las estructuras de datos
for index, row in parentFormsIds.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys85, cantonCode=cantonCode)

    poblacion_meta = ['Refugiados/as y migrantes', 'Comunidad de acogida']

    PRPI1_01_RM_NA = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    PRPI1_01_RM_NN = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    PRPI1_01_RM_OTR = int(df.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)
    dfDiversidad = osmosys.osmosys.getRefLgbtiDiscapacitadosValues(year=year, month=month, orgOsmosys=orgAcron,
                                                                   indicatorsIdsOmosys=indicatorIdsOsmosys85,
                                                                   cantonCode=cantonCode)
    PRPI1_01_RM_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)
    PRPI1_01_RM_DS = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'DISCAPACITADOS') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)

    dfCa = osmosys.osmosys.getCAValues(year=year, month=month, orgOsmosys=orgAcron,
                                       indicatorsIdsOmosys=indicatorIdsOsmosys85, cantonCode=cantonCode)
    PRPI1_01_CA_NA = int(dfCa.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    PRPI1_01_CA_NN = int(dfCa.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    PRPI1_01_CA_OTR = int(dfCa.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)

    PRPI1_01_CA_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'ECUADOR')].iloc[0].value_a)
    PRPI1_01_CA_DS = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'DISCAPACITADOS') & (
            dfDiversidad['country_of_origin'] == 'ECUADOR')].iloc[0].value_a)
    PRPI1_01_CUAL = commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                                               indicatorsIdsOmosys=indicatorIdsOsmosys85).iloc[
        0].value_a
    usd_transfer = int(osmosys.osmosys.getCBIBudget(year=year, month=month, orgOsmosys=orgAcron,
                                                    indicatorsIdsOmosys=indicatorIdsOsmosys85,
                                                    cantonCode=cantonCode).iloc[0].budget)
    subform = model.modelAI.SubFormPRPI1_01_85(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        covid='No',
        espacion_apoyo='No',
        poblacion_meta=poblacion_meta,
        modalidad_impl='cmo12q1kykwwosjka',  ## servicio
        mecanismos='c6gspwxkym0dbinm',
        transferencia='cib4u00kym0ixwew',
        usd_transfer=usd_transfer,
        PRPI1_01_RM_NA=PRPI1_01_RM_NA,
        PRPI1_01_RM_NN=PRPI1_01_RM_NN,
        PRPI1_01_RM_OTR=PRPI1_01_RM_OTR,
        PRPI1_01_RM_LGBT=PRPI1_01_RM_LGBT,
        PRPI1_01_RM_DS=PRPI1_01_RM_DS,
        PRPI1_01_CA_NA=PRPI1_01_CA_NA,
        PRPI1_01_CA_NN=PRPI1_01_CA_NN,
        PRPI1_01_CA_OTR=PRPI1_01_CA_OTR,
        PRPI1_01_CA_LGBT=PRPI1_01_CA_LGBT,
        PRPI1_01_CA_DS=PRPI1_01_CA_DS,
        PRPI1_01_CUAL=PRPI1_01_CUAL
    )

    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
print('changes to charge: ' + str(len(changesList)))

changes to charge: 6


In [46]:
## creo respaldo
changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)
print(finalJson)

{"changes": [{"formId": "clsgys9l4q3ommxdv", "recordId": "cl5j44bac000o8onmxlt127kg", "parentRecordId": "cl52k2v5o00oxl4nm7zg8ggs2", "fields": {"mes": "2022-06", "colltmgkykvhxgij6": "cjo64mykylzqu4p4r", "rmrp": "Si", "covid": "No", "espacion_apoyo": "No", "poblacion_meta": ["Refugiados/as y migrantes", "Comunidad de acogida"], "modalidad_impl": "cmo12q1kykwwosjka", "PRPI1_01_RM_NA": 1, "PRPI1_01_RM_NN": 1, "PRPI1_01_RM_OTR": 0, "PRPI1_01_RM_LGBT": 0, "PRPI1_01_RM_DS": 0, "PRPI1_01_CA_NA": 0, "PRPI1_01_CA_NN": 0, "PRPI1_01_CA_OTR": 0, "PRPI1_01_CA_LGBT": 0, "PRPI1_01_CA_DS": 0, "PRPI1_01_CUAL": "Ibarra: Se realiza el abordaje de 4 adolescentes (3 mujeres y 1 hombre), 3 casos fueron referidos por HIAS y 1 por la DP. Tulc\u00e1n: Durante el mes de junio se realizan dos ingresos  Se mantiene el acompa\u00f1amiento a 6 adolescentes durante el mes de junio incluidos los nuevos ingresos"}}, {"formId": "clsgys9l4q3ommxdv", "recordId": "cl5j44d8i000p8onmc3qgmv7f", "parentRecordId": "cl52k2v5o0

In [47]:
#open text file
try:
    os.mkdir(month)
except Exception:
    pass
text_file = open(os.path.join(month, "data_" + indicatorCodeAI + ".json"), "w")
#write string to file
n = text_file.write(finalJson)

#close file
text_file.close()
print(" creado respaldo: " + text_file.name)
newIdsDict = {"newIds": newIds}
newIdsDf = pd.DataFrame(newIdsDict)
newIdsDf.to_csv(os.path.join(month, "new_ids_" + indicatorCodeAI + ".csv"))

 creado respaldo: JUNIO\data_PRPI1_01.json


In [48]:
## envio a AI
if (test):
    print(
        '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')
else:
    try:
        if len(changesList) > 0:
            print('se envia a AI')
            client = Client(token=osmosys.osmosys.getToken(), base_url='https://www.activityinfo.org/resources')
            client.post_resource(path='update', body=finalJson)
            print(
                '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')
        else:
            print('nada que enviar a AI')
    except HTTPError as e:
        code = e.response.status_code
        print('error:')
        print(code)

        print(e)

--------------------------------------PRPI1_01------------------------------------------------------
